In [27]:
import numpy as np
import pandas as pd
import pyomo.opt
from pyomo.environ import *

In [28]:
depot_df = pd.read_csv('depot_info.csv')
facility_df = pd.read_csv('facility_position.csv')
cost_df = pd.read_csv('cost_matrix.csv',index_col=False)

In [29]:
num_facilities = facility_df.shape[0]
num_depots = depot_df.shape[0]
# num_depots = 200
# fixed_depot_costs = np.random.rand(num_depots)
fixed_depot_costs = depot_df['fixed_cost']
variable_depot_costs = cost_df.as_matrix()
# variable_depot_costs = np.random.rand(num_depots,num_facilities)

In [30]:
model = ConcreteModel()
model.name = 'UFL'
model.facility = range(num_facilities)
model.depot = range(num_depots)
model.x = Var(model.depot,model.facility,
             within=NonNegativeReals)
model.y = Var(model.depot,
             within=Binary)


In [31]:
model.capacity = ConstraintList()
for j in model.facility:
    model.capacity.add(
        sum(model.x[i,j] for i in model.depot ) >= facility_df.iloc[j].demand
    )
   

model.open_depot = ConstraintList()
for i in model.depot:
    model.open_depot.add(
        sum( model.x[i,j] for j in model.facility ) <= depot_df.iloc[i].capacity * model.y[i]
    )

In [32]:
def obj_rule(self):
    return sum( fixed_depot_costs[i]*model.y[i] for i in model.depot ) + \
        sum( variable_depot_costs[i,j]*model.x[i,j] for i in model.depot for j in model.facility )

model.obj = Objective(
    rule = obj_rule)

In [33]:
model.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('cplex')
results = solver.solve(model, tee = True)
model.solutions.load_from(results)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.8.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2017.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'c:\users\daniel~1\appdata\local\temp\tmpzuhjnj.cplex.log' open.
CPLEX> Problem 'c:\users\daniel~1\appdata\local\temp\tmpurfcqz.pyomo.lp' read.
Read time = 0.13 sec. (6.23 ticks)
CPLEX> Problem name         : c:\users\daniel~1\appdata\local\temp\tmpurfcqz.pyomo.lp
Objective sense      : Minimize
Variables            :  120201  [Nneg: 120001,  Binary: 200]
Objective nonzeros   :  120200
Linear constraints   :     801  [Less: 200,  Greater: 600,  Equal: 1]
  Nonzeros           :  240201
  RHS nonzeros       :     601

Variables            : Min LB: 0.0000000        Max UB: 1.000000       
Objecti

   2700   203   325157.7487    31   325513.6876   324882.0975    93774    0.19%
   2729   205   325329.4594    21   325513.6876   324882.0975    94477    0.19%
   2769   230   325316.8023    26   325513.6876   324882.0975    95701    0.19%
   2802   250        cutoff         325513.6876   324882.0975    99139    0.19%
   2835   260   325355.5902    28   325513.6876   324882.0975   100121    0.19%
Elapsed time = 129.94 sec. (81744.55 ticks, tree = 7.68 MB, solutions = 5)
   2872   280   325376.9834    27   325513.6876   324882.0975   102236    0.19%
   2907   307   325210.9033    29   325513.6876   324882.0975   105299    0.19%
   2961   315   325145.3924    25   325513.6876   324882.0975   105856    0.19%
   3000   362   325091.2932    30   325513.6876   324882.0975   108736    0.19%
   3045   373   324993.7922    41   325513.6876   324882.0975   109229    0.19%
   3084   421   325405.5435    26   325513.6876   324882.0975   111994    0.19%
   3129   444   325335.6926    22   325513.68

In [8]:
open_depots = []
for i in model.depot:
    if model.y[i].value==1:
        open_depots.append(i)
print(open_depots)

[0, 1, 4, 7, 9, 13, 20, 21, 32, 42, 48, 58, 73, 76, 82, 89, 91, 94, 95, 98]


In [9]:
for i in model.depot:
    for j in model.facility:
        if model.x[i,j].value!=0:
            print(model.x[i,j].value)

47.5886573074
26.2010899142
10.0
10.0
28.1394991059
34.3085807228
15.8815835814
10.0
13.9681088455
48.0518816083
36.33189915
33.2488475916
17.8436397026
26.1082310964
14.7510849289
9.45411365251
40.4342245067
16.492738019
35.6075473951
10.5640941898
10.0
10.0
47.7069157267
38.6401410904
19.0045828925
10.0
20.2867266286
32.9482717286
24.6756021025
47.1611251101
14.1395126933
10.0
33.6109283021
10.0
10.0
40.4648213084
10.0
10.147374948
10.0
38.9086763578
32.6298111461
10.0
14.9910028391
34.0661893936
23.1386761847
29.2655196356
27.8155020138
34.0555683771
10.0
25.5911336857
10.0
25.4930565142
35.8608651022
15.688275213
40.3142936621
49.672678854
37.3184615355
43.8682002665
10.0
42.3296274461
22.3688140139
10.0
19.6896672179
29.1078339132
14.3034077759
9.50913624825
10.0
27.036128808
19.0503635304
48.5933074123
49.7632701009
37.7453238043
27.6264711717
28.4274482309
24.1294455139
10.0245304366
10.0
16.8675960697
10.0
3.05516294758
33.508043
13.4367940524
22.7677483537
40.6118138981
26.088

In [11]:
print(type(model.capacity))

<class 'pyomo.core.base.constraint.ConstraintList'>


In [12]:
print ("Duals")
from pyomo.core import Constraint
for c in model.component_objects(Constraint, active=True):
    print ("   Constraint",c)
    cobject = getattr(model, str(c))
    print(cobject)
    for index in cobject:
        print(cobject[index])
        print ("      ", index, model.dual[cobject[index]])

Duals
('   Constraint', <pyomo.core.base.constraint.ConstraintList object at 0x000000000BD17D30>)
capacity
capacity[1]


KeyError: "Component with id '197511304': capacity[1]"